# QA EditAttemtpStep Content Translation Integration

[Task](https://phabricator.wikimedia.org/T270636)

# Background

Content Translation uses the visual editor codebase for its editing surface. We'd like to explicitly tag and mark events being logged in EAS and VEFU schemas as being emitted by ContentTranslation so they can be differentiated from other VE related edits.

A patch was deployed on 19 August 2021 to add `integration=contenttranslation` to the EditAttemptStep schema.

# Purpose

Peform post-deployment QA of aggregate data logged in EditAttemptStep to ensure that we're  seeing events getting logged with integration==contenttranslation and numbers appear as expected.

In [11]:
# load required packages
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); library(glue); library(lubridate); library(scales)
})

# QA 

In [12]:
query <-

"SELECT
    CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) as `date`,
    event.action AS action,
    event.init_type AS init_type,
    event.platform As platform,
    event.integration AS integration,
    event.user_id AS user_id,
    event.editing_session_id AS session_id
FROM event.editattemptstep
WHERE year = 2021
AND month = 08
AND Day >= 16;
"

In [13]:
edit_data <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [14]:
# reformat as date
edit_data$date <- as.Date(edit_data$date, format = "%Y-%m-%d")

In [15]:
# check for when content translation events started logging

ct_events_daily <- edit_data %>%
    ##filter(integration == 'contenttranslation') %>%
    group_by(date, integration)%>%
    summarise(num_events = n())

ct_events_daily 

`summarise()` regrouping output by 'date' (override with `.groups` argument)



date,integration,num_events
<date>,<chr>,<int>
2021-08-16,discussiontools,10555
2021-08-16,page,771480
2021-08-17,discussiontools,11463
2021-08-17,page,776002
2021-08-18,discussiontools,11089
2021-08-18,page,774257
2021-08-19,discussiontools,9949
2021-08-19,page,759417
2021-08-20,discussiontools,10400


In [ ]:
No contenttranslation events recorded to date